In [1]:
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd

# Number of units built in each census tract

In [2]:
total_units = pd.read_csv("./data/bayarea_total_units_JG.csv")
total_units.head(5)

,FIPS13,YearBuilt,TotalUnits
0,60013522021051,2001,1
1,60013522021051,2009,1
2,60013530012030,2000,1
3,60013530012041,2004,1
4,60013530012041,2009,1


In [3]:
total_units = total_units.rename(columns={'FIPS13': 'FIPS13', 'YearBuilt': 'year', 'TotalUnits': 'total_units'})

In [4]:
geoid_10 = total_units['FIPS13'].astype(str)
geoid_10 = geoid_10.str.slice(stop=10)
geoid_10.head(5)

0    6001352202
1    6001352202
2    6001353001
3    6001353001
4    6001353001
Name: FIPS13, dtype: object

In [5]:
total_units_geoid = total_units.drop(columns=['FIPS13'])
total_units_geoid['GEO_ID_10'] = geoid_10
total_units_geoid.head(5)

,year,total_units,GEO_ID_10
0,2001,1,6001352202
1,2009,1,6001352202
2,2000,1,6001353001
3,2004,1,6001353001
4,2009,1,6001353001


In [6]:
total_units_geoid['GEO_ID_10'] = total_units_geoid['GEO_ID_10'].str.zfill(11)
total_units_geoid.head(5)

,year,total_units,GEO_ID_10
0,2001,1,06001352202
1,2009,1,06001352202
2,2000,1,06001353001
3,2004,1,06001353001
4,2009,1,06001353001


Changed county for census_tract 35511 to 013 (Contra Costa County).

In [7]:
total_units_geoid.iloc[7, 2] = '0601335511'

In [8]:
total_units_by_tract_by_year = total_units_geoid.groupby(
        by=['GEO_ID_10','year'], 
        axis=0, 
        as_index=False) \
    .sum()
total_units_by_tract_by_year.head(5)

,GEO_ID_10,year,total_units
0,06001352202,2001,1
1,06001352202,2009,1
2,06001353001,2000,1
3,06001353001,2004,1
4,06001353001,2009,1


In [9]:
total_units_2010_2019 = total_units_by_tract_by_year[total_units_by_tract_by_year['year']>=2010]
total_units_2010_2019.head(5)

,GEO_ID_10,year,total_units
17,06001400100,2010,9
18,06001400100,2011,3
19,06001400100,2012,1
20,06001400100,2013,1
21,06001400100,2014,1


# Converting tracts to zip codes

In [10]:
tract_to_zip_list = []

for i in range(0, 10):
    year = f'201{i}'
    df = pd.read_excel(f'./data/TRACT_ZIP_12{year}.xlsx')
    df.columns = df.columns.str.lower()
    df['tract'] = df['tract'].astype(str).str.zfill(11)
    df['year'] = int(year)
    df = df.drop(columns=['bus_ratio', 'oth_ratio', 'tot_ratio'])
    df = df.rename(columns={'res_ratio': 'resident'})
    tract_to_zip_list.append(df)

In [11]:
tract_to_zip = pd.concat(tract_to_zip_list, axis=0)
tract_to_zip.head(5)

,tract,zip,resident,year
0,01001020100,36067,1.000000,2010
1,01001020200,36008,0.027225,2010
2,01001020200,36067,0.972775,2010
3,01001020300,36067,1.000000,2010
4,01001020400,36066,0.962221,2010


In [12]:
total_units_per_tract = pd.merge(total_units_2010_2019, 
                               tract_to_zip, 
                               left_on=['GEO_ID_10', 'year'], 
                               right_on=['tract', 'year'])
total_units_per_tract.head(5)

,GEO_ID_10,year,total_units,tract,zip,resident
0,06001400100,2010,9,06001400100,94611,0.042201
1,06001400100,2010,9,06001400100,94618,0.287867
2,06001400100,2010,9,06001400100,94704,0.034665
3,06001400100,2010,9,06001400100,94705,0.635267
4,06001400100,2010,9,06001400100,94720,0.000000


In [13]:
total_units_per_tract['units_built'] = total_units_per_tract['total_units'] * total_units_per_tract['resident']
total_units_per_tract.head(5)

,GEO_ID_10,year,total_units,tract,zip,resident,units_built
0,06001400100,2010,9,06001400100,94611,0.042201,0.379804
1,06001400100,2010,9,06001400100,94618,0.287867,2.590803
2,06001400100,2010,9,06001400100,94704,0.034665,0.311982
3,06001400100,2010,9,06001400100,94705,0.635267,5.717403
4,06001400100,2010,9,06001400100,94720,0.000000,0.000000


In [14]:
total_units_per_zip = total_units_per_tract[['year', 'zip', 'units_built']]
total_units_per_zip.head(5)

,year,zip,units_built
0,2010,94611,0.379804
1,2010,94618,2.590803
2,2010,94704,0.311982
3,2010,94705,5.717403
4,2010,94720,0.000000


In [15]:
total_units_by_zip = total_units_per_zip.groupby(by=['zip', 'year'], axis=0, as_index=False).sum()
total_units_by_zip.head(5)

,zip,year,units_built
0,94002,2010,3.012672
1,94002,2011,4.007986
2,94002,2012,3.152709
3,94002,2013,3.961284
4,94002,2014,3.000000


In [16]:
total_units_by_zip['units_built'] = total_units_by_zip['units_built'].apply(np.round).astype(int)
total_units_by_zip.head(5)

,zip,year,units_built
0,94002,2010,3
1,94002,2011,4
2,94002,2012,3
3,94002,2013,4
4,94002,2014,3


# Housing Prices

In [17]:
zillow = pd.read_csv('./data/zillow.csv')

In [18]:
zillow.head(5)

,RegionID,SizeRank,RegionName,RegionType,StateName,State,City,Metro,CountyName,1996-01-31,...,2020-06-30,2020-07-31,2020-08-31,2020-09-30,2020-10-31,2020-11-30,2020-12-31,2021-01-31,2021-02-28,2021-03-31
0,61639,0,10025,Zip,NY,NY,New York,New York-Newark-Jersey City,New York County,227153.0,...,1091641.0,1087154.0,1085121.0,1086397.0,1082450.0,1078831.0,1074640.0,1066881.0,1069011.0,1073214.0
1,84654,1,60657,Zip,IL,IL,Chicago,Chicago-Naperville-Elgin,Cook County,212230.0,...,497724.0,499152.0,501262.0,504040.0,506695.0,509141.0,511452.0,513457.0,515542.0,516747.0
2,61637,2,10023,Zip,NY,NY,New York,New York-Newark-Jersey City,New York County,200946.0,...,998619.0,998643.0,1001866.0,1013305.0,1016169.0,1014689.0,1009092.0,1011561.0,1021442.0,1029770.0
3,91982,3,77494,Zip,TX,TX,Katy,Houston-The Woodlands-Sugar Land,Harris County,201011.0,...,339450.0,340170.0,341602.0,343191.0,345709.0,348800.0,353185.0,357460.0,362468.0,367829.0
4,84616,4,60614,Zip,IL,IL,Chicago,Chicago-Naperville-Elgin,Cook County,283626.0,...,635967.0,637199.0,638751.0,641622.0,644350.0,647291.0,650118.0,652578.0,655761.0,658233.0


In [19]:
zillow = zillow.drop(columns=['RegionID', 
                              'SizeRank', 
                              'RegionType', 
                              'StateName', 
                              'State', 
                              'City', 
                              'Metro', 
                              'CountyName'])
zillow.head(5)

,RegionName,1996-01-31,1996-02-29,1996-03-31,1996-04-30,1996-05-31,1996-06-30,1996-07-31,1996-08-31,1996-09-30,...,2020-06-30,2020-07-31,2020-08-31,2020-09-30,2020-10-31,2020-11-30,2020-12-31,2021-01-31,2021-02-28,2021-03-31
0,10025,227153.0,225738.0,225814.0,225790.0,226306.0,226934.0,226823.0,227619.0,227965.0,...,1091641.0,1087154.0,1085121.0,1086397.0,1082450.0,1078831.0,1074640.0,1066881.0,1069011.0,1073214.0
1,60657,212230.0,212073.0,211661.0,211524.0,210956.0,211141.0,210927.0,211539.0,212435.0,...,497724.0,499152.0,501262.0,504040.0,506695.0,509141.0,511452.0,513457.0,515542.0,516747.0
2,10023,200946.0,200989.0,201100.0,201816.0,202857.0,203710.0,203994.0,204078.0,204399.0,...,998619.0,998643.0,1001866.0,1013305.0,1016169.0,1014689.0,1009092.0,1011561.0,1021442.0,1029770.0
3,77494,201011.0,201259.0,201062.0,199869.0,198728.0,198060.0,198825.0,198512.0,197857.0,...,339450.0,340170.0,341602.0,343191.0,345709.0,348800.0,353185.0,357460.0,362468.0,367829.0
4,60614,283626.0,283367.0,282845.0,282702.0,282021.0,282109.0,281613.0,282177.0,283263.0,...,635967.0,637199.0,638751.0,641622.0,644350.0,647291.0,650118.0,652578.0,655761.0,658233.0


In [20]:
zillow_q4 = zillow[[zillow.columns[0]] + [col for col in zillow.columns if '-12-' in col]]
zillow_q4.head(5)

,RegionName,1996-12-31,1997-12-31,1998-12-31,1999-12-31,2000-12-31,2001-12-31,2002-12-31,2003-12-31,2004-12-31,...,2011-12-31,2012-12-31,2013-12-31,2014-12-31,2015-12-31,2016-12-31,2017-12-31,2018-12-31,2019-12-31,2020-12-31
0,10025,232288.0,249559.0,297409.0,365222.0,457899.0,537778.0,590102.0,706128.0,811054.0,...,917022.0,952211.0,1044071.0,1183382.0,1260988.0,1299640.0,1207579.0,1268144.0,1184383.0,1074640.0
1,60657,216901.0,220413.0,248569.0,282386.0,326790.0,361820.0,382506.0,397071.0,419766.0,...,398085.0,396617.0,450111.0,460604.0,467852.0,491532.0,496357.0,503865.0,491195.0,511452.0
2,10023,206659.0,229875.0,251912.0,279682.0,332722.0,382766.0,424800.0,488220.0,556784.0,...,766144.0,792216.0,873578.0,971943.0,1027807.0,1046132.0,1033575.0,1023463.0,966318.0,1009092.0
3,77494,199658.0,215038.0,207634.0,219026.0,227440.0,228291.0,228549.0,235857.0,247465.0,...,262289.0,269758.0,301976.0,330015.0,340156.0,331932.0,333138.0,337393.0,337232.0,353185.0
4,60614,289462.0,291350.0,325137.0,364159.0,417792.0,456994.0,484111.0,498849.0,525347.0,...,501378.0,503722.0,571893.0,591076.0,611218.0,635666.0,641008.0,650355.0,634193.0,650118.0


In [21]:
zillow_2010_2019 = zillow_q4[[zillow.columns[0]] + 
                             [col for col in zillow_q4.columns if '2010' <= col[:4] < '2020']]
zillow_2010_2019.head(5)

,RegionName,2010-12-31,2011-12-31,2012-12-31,2013-12-31,2014-12-31,2015-12-31,2016-12-31,2017-12-31,2018-12-31,2019-12-31
0,10025,907964.0,917022.0,952211.0,1044071.0,1183382.0,1260988.0,1299640.0,1207579.0,1268144.0,1184383.0
1,60657,412886.0,398085.0,396617.0,450111.0,460604.0,467852.0,491532.0,496357.0,503865.0,491195.0
2,10023,725539.0,766144.0,792216.0,873578.0,971943.0,1027807.0,1046132.0,1033575.0,1023463.0,966318.0
3,77494,265354.0,262289.0,269758.0,301976.0,330015.0,340156.0,331932.0,333138.0,337393.0,337232.0
4,60614,518787.0,501378.0,503722.0,571893.0,591076.0,611218.0,635666.0,641008.0,650355.0,634193.0


In [22]:
zillow_years = []

for i in range(10):
    year = f'201{i}'
    df = zillow_2010_2019[['RegionName', f'{year}-12-31']]
    df = df.rename(columns={'RegionName': 'zip', f'{year}-12-31': 'price'})
    df['year'] = int(year)
    zillow_years.append(df)

zillow_by_year = pd.concat(zillow_years, axis=0)
zillow_by_year.head(5)

,zip,price,year
0,10025,907964.0,2010
1,60657,412886.0,2010
2,10023,725539.0,2010
3,77494,265354.0,2010
4,60614,518787.0,2010


# Housing Prices with Housing Units built

In [23]:
data = pd.merge(total_units_by_zip, zillow_by_year, left_on=['zip', 'year'], right_on=['zip', 'year'])
data

,zip,year,units_built,price
0,94002,2010,3,887752.0
1,94002,2011,4,877332.0
2,94002,2012,3,995829.0
3,94002,2013,4,1243884.0
4,94002,2014,3,1363204.0
...,...,...,...,...
2151,95694,2013,1,309138.0
2152,95694,2014,2,346545.0
2153,95694,2015,1,375418.0
2154,95694,2016,0,412691.0


In [24]:
data_2010 = data[data['year'] == 2010]
data_2010.head(5)

,zip,year,units_built,price
0,94002,2010,3,887752.0
7,94005,2010,4,566928.0
13,94010,2010,30,1318145.0
20,94014,2010,209,540374.0
29,94015,2010,1,531530.0


In [25]:
data_2010.to_csv('./data_out/data_2010.csv')

FileNotFoundError: [Errno 2] No such file or directory: './data_out/data_2010.csv'

In [ ]:
print(data.head(10).to_html(index=False))